In [ ]:
import numpy as np 
import pandas as pd

# data visualisation
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns 

import missingno as msno #visualises missing values
from wordcloud import WordCloud #to find the most frequent words 

In [ ]:
#data that was scraped from source
scrape = "https://raw.githubusercontent.com/globaldothealth/monkeypox/main/latest.csv"

#### Pre-Processing(data cleaning):
- loading
- peeking
- general info

In [ ]:
#loading data 
df = pd.read_csv(scrape)

In [ ]:
df.head()

In [ ]:
df.tail() #addressed results a few lines down

In [ ]:
#number of countries with monkeypox cases
len(df["Country"].unique())

In [ ]:
df.info()

In [ ]:
#to see the number of missing values in the dataset

df.isnull().sum() 

##### Although there a quite a few missing values in some columns, these values are not of much importance as they do not affect the integrity of core information such as the number of confirmed cases.

### Exploratory Data Analysis (EDA)

In [ ]:
df["Country"].value_counts()

In [ ]:
#function that plots the number of monkeypox cases per country (matplotlib & seaborn)
def countrycases():
    plt.figure(figsize=(50,8))
    sns.countplot(x="Country", data=df)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countrycases()

In [287]:
#due to the USA's high monkeypox count i will further breakdown its cases based on cities

#REMEMBER TO REWRITE YOUR CODE, ITS NOT PRINTING IT IN THE FORMAT YOU WANT!!!



df[df["Country"] == "United States"].groupby("City")["Status"].size()

City
Anchorage         2
Austin            1
Boston            1
Chicago           7
Cincinnati        2
Cleveland         1
Dallas            1
Denver            1
Forest Park       1
Houston           2
Kansas City       2
Louisville        2
Milwaukee         1
Nashville         3
Philadelphia      2
Sacramento        1
San Francisco    20
San Francsico    16
St. Louis         1
Washington DC     1
Name: Status, dtype: int64